In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libraries
import gzip
import numpy as np
import pandas as pd
import string

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
# define X, y and test
X = train['excerpt']
y = train['target']
test_text = test["excerpt"]

In [ ]:
# lower the text
X = X.str.lower()
test_text = test_text.str.lower()

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
X = X.apply(ps.stem)
test_text = test_text.apply(ps.stem)

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
#nltk.download('wordnet')

wnl = WordNetLemmatizer()
X = X.apply(wnl.lemmatize)
test_text = test_text.apply(wnl.lemmatize)

In [ ]:
# # lemmatize the text
# from nltk.stem import WordNetLemmatizer
# import nltk
# # nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()
# def lemmatize_sentences(sentence):
#     tokens = sentence.split()
#     stemmed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
#     return ' '.join(stemmed_tokens)

# X = X.apply(lemmatize_sentences)
# test_text = test_text.apply(lemmatize_sentences)

In [ ]:
import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

X = X.apply(lambda text: remove_stopwords(text))
test_text = test_text.apply(lambda text: remove_stopwords(text))

In [ ]:
X_new = []
for index, value in X.items():
    X_new.append(value)
    
y_new = []
for index, value in y.items():
    y_new.append(value)
    
real_test = []
for index, value in test_text.items():
    real_test.append(value)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn import linear_model

# m = RandomForestRegressor()
m = linear_model.Lasso(alpha=0.01)


In [ ]:
# vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))
# m.fit(vectorizer.fit_transform(X_new + real_test)[:len(X_new)], y_new)

In [ ]:
# EMBEDDING_FILE = '../input/glovefile/glove.6B.100d.txt'

# embeddings = {}
# for o in open(EMBEDDING_FILE):
#     word = o.split(" ")[0]
#     # print(word)
#     embd = o.split(" ")[1:]
#     embd = np.asarray(embd, dtype='float32')
#     # print(embd)
#     embeddings[word] = embd

In [ ]:
def get_feature_vectors(sentence):
    words = sentence.split()
    feature_vec = np.zeros((100,),dtype="float32")
    i=0
    for word in words:
        try:
            feature_vec = np.add(feature_vec, embeddings.get(word))
        except:
            i = i + 1
    if len(words) > 0:
        feature_vec = np.divide(feature_vec, len(words)- i)
    return feature_vec

In [ ]:
real_test_vectors = np.array([get_feature_vectors(sentence) for sentence in real_test])
train_vectors_full = np.array([get_feature_vectors(sentence) for sentence in X_new])

In [ ]:
# m.fit(train_vectors_full, y_new)

In [ ]:
import transformers
import torch
from transformers import BertModel, BertTokenizer

In [ ]:
model = BertModel.from_pretrained('../input/huggingface')
tokenizer = BertTokenizer.from_pretrained('../input/huggingface')

In [ ]:
def get_feature_vectors(sentence):
    token = tokenizer.encode(sentence)
    sent_tensor = torch.tensor([token])
    token_len = len(token)

    # Xây dựng attention_mask, ở đây là mảng toàn 1 vì ta quan tâm đến all các token
    attention_mask = torch.tensor([np.ones(token_len)])

    # Trích đặc trưng
    with torch.no_grad():
        last_hidden_states = model(input_ids = sent_tensor, attention_mask = attention_mask)

    # In kích thước và giá trị đặc trưng ra màn hình
    return last_hidden_states[0][:, 0, :][0].numpy()


In [ ]:
real_test_vectors = [get_feature_vectors(sentence) for sentence in real_test]
train_vectors_full = [get_feature_vectors(sentence) for sentence in X_new]

In [ ]:
m.fit(train_vectors_full, y_new)


In [ ]:
pred_test = m.predict(real_test_vectors)
pred_test_list = [i for i in pred_test]
pred_test_list

In [ ]:
submission = pd.DataFrame({'id' : test['id'], 'target' : pred_test_list})
submission.to_csv('/kaggle/working/submission.csv', index=False)
submission.head(7)